In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import statistics
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
plt.style.use('seaborn')

In [3]:
'''
- check tensorflow and keras version
'''
print(f'Tensorflow GPU Version: {tf.__version__}')
print(f'Eager Execution is: {tf.executing_eagerly()}')
print(f'Keras Version: {tf.keras.__version__}')

Tensorflow GPU Version: 2.0.0
Eager Execution is: True
Keras Version: 2.2.4-tf


In [4]:
'''
- check GPU 
'''
var = tf.Variable([3,3])
if tf.test.is_gpu_available():
    print('Running on GPU')
else:
    print('Runing on CPU')

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Running on GPU


Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

# MLP model

In [10]:
'''
- MLP layer
'''
class MLP(tf.keras.layers.Layer):
    def __init__(self, units=3, activation=None, trainable=True, name=None, dtype=tf.float32, **kwargs):
        super(MLP, self).__init__( name=name, trainable = trainable, dtype=dtype, **kwargs)
        self.units = units
        self.__activation_name = activation
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),initializer='random_normal',)
        self.b = self.add_weight(shape=(self.units,),initializer='random_normal')
        
    def get_config(self):
        config_dic ={
            'units':self.units,
            'activation': self.__activation_name, 
            'trainable_weights & bias':self.trainable_weights,
            'non-trainable_weights & bias':self.non_trainable_weights,
        }
        config = super(MLP,self).get_config()
        config.update(config_dic)
        return config
    
    def call(self, inputs, training = None):
        if tf.rank(inputs)==1:
            inputs = tf.keras.backend.expand_dims(inputs, axis=0)
        linear_combination = tf.matmul(inputs, self.w)+self.b 
        return self.activation(linear_combination)

# Multi-layer Perceptron Model

In [7]:
'''
- subclass model
'''
class MLP_Model(tf.keras.Model):
    def __init__(self, **kwargs):
        super(MLP_Model, self).__init__(self, **kwargs)
        self.mlp1 = MLP(256, activation='sigmoid')
        self.mlp2 = MLP(128, activation='sigmoid')
        self.mlp3 = MLP(10, acitivation='softmax')
        
    def call(self, inputs):
        opt = self.mlp1(inputs)
        opt = self.mlp2(opt)
        opt = self.mlp3(opt)
        return opt

# Logistic Regression Model

In [12]:
'''
- logistic regression
- perform binary classification with multi-labels
'''
class LogisticRegression(tf.keras.Model):
    def __init__(self,labels=1, **kwargs):
        super(LogisticRegression, self).__init__(self, **kwargs)
        self.label = labels
        self.lglayer = MLP(labels, activation='sigmoid')
    
    def call(self, inputs):
        opt= self.lglayer(inputs)
        return opt

In [13]:
lg_model = LogisticRegression(labels=3)

# Weighted KNN

In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
"""
- see WKNN.py for coding details
- weighted KNN algorithm refers to the paper published by Klaus Hechenbichler and Klaus Schliep in 2004
- paper link: https://epub.ub.uni-muenchen.de/1769/1/paper_399.pdf
- the paper provides 3 similarity calculations and 8 weight transfer kernels, but in this model only basics are
implemented
    similarity calculation: Euclidean Distance
    weight transfer calculation: Inversion Kernel
- class arguments:
    train_data: numpy array, tensor data,
    train_labels: numpy array, tensor data,
    k_value: must be an integer
- call method:
    inputs: single data vector or a batch of data
- return:
    if the input is a single data, it will return a classification
    if the input is a batch of data, it will return a classification list
- initial unit test members work with test_WKNN.py and comment out @tf.function
"""

'\n- see WKNN.py for coding details\n- weighted KNN algorithm refers to the paper published by Klaus Hechenbichler and Klaus Schliep in 2004\n- paper link: https://epub.ub.uni-muenchen.de/1769/1/paper_399.pdf\n- the paper provides 3 similarity calculations and 8 weight transfer kernels, but in this model only basics are\nimplemented\n    similarity calculation: Euclidean Distance\n    weight transfer calculation: Inversion Kernel\n- class arguments:\n    train_data: numpy array, tensor data,\n    train_labels: numpy array, tensor data,\n    k_value: must be an integer\n- call method:\n    inputs: single data vector or a batch of data\n- return:\n    if the input is a single data, it will return a classification\n    if the input is a batch of data, it will return a classification list\n- initial unit test members work with test_WKNN.py and comment out @tf.function\n'

In [7]:
iris = datasets.load_iris()
print('data: \n', iris['data'][0:10,:])
print('target: \n', iris['target'][0:10])
data, test_data, labels, test_labels = train_test_split(iris['data'], iris['target'], test_size=0.3, random_state=1)
print('data size: ', len(data))
print('label size: ', len(labels))
print('test data size: ', len(test_data))
print('test label size: ', len(test_labels))

data: 
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]
target: 
 [0 0 0 0 0 0 0 0 0 0]
data size:  105
label size:  105
test data size:  45
test label size:  45


In [8]:
wknn = WKNN(data, labels)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# Unsupervised Learning Model